In [1]:
import csv, networkx as nx, operator

In [2]:
source = 'dolphins'

In [3]:
graph = nx.Graph()
reader = csv.reader(open(f'./PageRank/data/{source}.csv', 'r'), delimiter=',')
data = [row for row in reader]

nodes = set([row[0] for row in data])
edges = [(row[0], row[2]) for row in data]
num_nodes = len(nodes)
s = 0.85
initial_rank = 1/float(num_nodes)

graph.add_nodes_from(nodes, rank=initial_rank)
graph.add_edges_from(edges)
pages = {} # 각 아이템은 다음과 같은 형식으로 값을 가진다. {page_id: [pagerank, neighbors]}

for page_id, node in graph.nodes(data=True):
    neighbors = graph.neighbors(page_id)
    neighbors_ids = [neighbor for neighbor in neighbors]
    value = [initial_rank, neighbors_ids]
    pages[page_id] = value

# 맵 리듀스 방식으로 구현한 페이지 랭크 알고리즘

In [4]:
def mapper(page_id, value):
    neighbors = graph.neighbors(page_id)
    outputs = [(page_id, value)]
    # r_i / d_out(i) 계산
    out_pagerank = value[0] / len(value[1])
    for neighbor in neighbors:
        outputs.append((neighbor, out_pagerank))
    return outputs

In [5]:
def reducer(page_id, values):
    rank_sum = 0
    output = [0, []]
    for value in values:
        if type(value) == list:
            output = value
        else:
            rank_sum += value
    # new pagerank 계산
    output[0] = (1-s) * (1/num_nodes) + s*rank_sum
    return (page_id, output)

In [6]:
def map_reduce(dataset, mapper, reducer):
    intermediate = []
    for (key, value) in dataset.items():
        intermediate.extend(mapper(key, value))

    groups = {}
    for (key, value) in intermediate:
        if key not in groups:
            groups[key] = []
        groups[key].append(value)
    output = {}

    for (key, values) in groups.items():
        (new_key, new_value) = reducer(key, values)
        output[new_key] = new_value
        
    return output

In [7]:
result = {}
for i in range(30):
    result = map_reduce(pages, mapper, reducer)

ranks = {}
for key, value in result.items():
    ranks[key] = value[0]

sorted_ranks = sorted(ranks.items(), key=operator.itemgetter(1), reverse=True)

for i in range(min(10, len(ranks))):
    print(sorted_ranks[i])

('Grin', 0.03214349344215876)
('Jet', 0.031729872609133285)
('Trigger', 0.031298461173290625)
('Web', 0.030097132110645918)
('SN4', 0.02987447822639584)
('Topless', 0.02951328555911399)
('Scabs', 0.02842221290110614)
('Patchback', 0.02645774552453914)
('Gallatin', 0.026158554472606777)
('Beescratch', 0.024651663857178114)


# Loop 방식으로 구현한 페이지 랭크 알고리즘

In [8]:
V = num_nodes
d = s
for _ in range(10):
    for key, node in graph.nodes(data=True):
        rank_sum = 0.0
        curr_rank = node.get('rank')
        neighbors = graph[key]
        for n in neighbors:
            if ranks[n] is not None:
                outlinks = len(list(graph.neighbors(n)))
                rank_sum += (1 / float(outlinks)) * ranks[n]
    
        # actual page rank compution
        ranks[key] = ((1 - float(d)) * (1/float(V))) + d*rank_sum

In [9]:
sorted_ranks = sorted(ranks.items(), key=operator.itemgetter(1), reverse=True)

for i in range(min(10, len(ranks))):
    print(sorted_ranks[i])

('Grin', 0.032144467799220436)
('Jet', 0.03172817443472615)
('Trigger', 0.03129933558066226)
('Web', 0.030095411329659486)
('SN4', 0.029875320707340957)
('Topless', 0.029514184521112843)
('Scabs', 0.028423051321498843)
('Patchback', 0.02645852919569144)
('Gallatin', 0.026156910346649482)
('Beescratch', 0.02465074097276958)


두 방식이 같은 결과를 내는지 확인하였다. lesmis와 같이 약간의 값 차이는 있지만, 전체적인 순서는 동일함을 알 수 있다.